In [1]:
import os
os.chdir("/home/dephinate/ASU/DL/MisterRetriveRite/")
! pwd

/home/dephinate/ASU/DL/MisterRetriveRite


Install Modules

In [2]:
# ! pip install python-box==6.0.2
# ! pip install ensure==1.0.2
# ! pip install -e .


In [3]:
from misterRetriveRite.config.configurations import ConfigurationManager


In [4]:
config = ConfigurationManager()

[2024-03-05 13:20:45,601,INFO,common,created directory at: artifacts]


In [5]:
print(config.get_model_config())
print(config.get_vectorization_config())

ModelConfig(model_name='gpt-3.5-turbo-instruct', temperature=0.9, max_tokens=500, chunk_size=2000, chunk_overlap=100)
[2024-03-05 13:20:46,235,INFO,common,created directory at: artifacts/vector_db]
VectorizationConfig(root_dir='artifacts/vector_db', encoder_name='all-mpnet-base-v2', model_ckpt='None', data_path='None', k='None', num_of_cells='None', nprobe='None')


Data Loader

In [6]:
from langchain.document_loaders import UnstructuredURLLoader
class DataLoader():
    def __init__(self) -> None:
        pass
    def load_from_url(self,urls: list):
        loader = UnstructuredURLLoader(urls=urls)
        data = loader.load()
        return data

[2024-03-05 13:20:50,547,INFO,utils,Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-03-05 13:20:50,548,INFO,utils,NumExpr defaulting to 8 threads.]


Data Splitter

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
class Splitter():
    def __init__(self,data) -> None:
        self.data = data
        pass
    def split_recursive(self, chunk_size:int,chunk_overlap:int,sperators:list[str]):
        Splitter = RecursiveCharacterTextSplitter(
            separators=sperators,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        splits = Splitter.split_documents(self.data)
        return splits

Vectorization

In [8]:
from langchain.vectorstores import FAISS
from misterRetriveRite.utils.common import pickel_dump

class Vectorizer():
    def __init__(self,data) -> None:
        self.data = data
    def build_vectorindex_with_faiss_and_openai(self, save_to_local:bool,file_path:None):
        from langchain.embeddings import OpenAIEmbeddings
        embeddings = OpenAIEmbeddings()
        vectorindex = FAISS.from_documents(self.data, embeddings)
        if save_to_local:
            pickel_dump(file_path=file_path,data=vectorindex)
            return vectorindex
        return vectorindex
    
    def build_vectorindex_with_faiss_and_huggingface(self, model_name: str,save_to_local:bool,file_path:None):
        from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
        embeddings = HuggingFaceBgeEmbeddings(model_name = model_name)
        vectorindex = FAISS.from_documents(self.data, embeddings)
        if save_to_local:
            pickel_dump(file_path=file_path,data=vectorindex)
            return vectorindex
        return vectorindex


